# Miljødataanalyse - Del 1, Oppgave 2.

In [ ]:
# Sjekk directory
import os
print(os.getcwd())

In [34]:
#Markus eksperimenterer med en alternativ API her.

# Test: Importere og bruke API fra openweathermap.org

import requests
import pandas as pd
import json
import matplotlib.pyplot as plt # for plotting
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('API_KEY')

city = 'Oslo'

url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'

res = requests.get(url)
data = res.json()


humidity = data['main']['humidity']
pressure = data['main']['pressure']
wind = data['wind']['speed']
description = data['weather'][0]['description']
temp = data['main']['temp']

print('Temperature:',temp,'°C')
print('Wind:',wind)
print('Pressure: ',pressure)
print('Humidity: ',humidity)
print('Description:',description)


df = pd.json_normalize(data)

print(df)

# df_data = pd.DataFrame(data)

# df_data.drop_duplicates(inplae=True)
# df_no_duplicates = df_data.drop_duplicates()

# print(df_data)




Temperature: 13.33 °C
Wind: 3.55
Pressure:  1005
Humidity:  60
Description: overcast clouds
                                             weather      base  visibility  \
0  [{'id': 804, 'main': 'Clouds', 'description': ...  stations       10000   

           dt  timezone       id  name  cod  coord.lon  coord.lat  ...  \
0  1743081161      3600  3143244  Oslo  200    10.7461    59.9127  ...   

   main.grnd_level  wind.speed  wind.deg  wind.gust  clouds.all  sys.type  \
0              981        3.55       225       7.16         100         2   

   sys.id  sys.country  sys.sunrise  sys.sunset  
0   46775           NO   1743051434  1743097637  

[1 rows x 27 columns]


In [35]:
#Nika
#open-meteo.com
#Historical forecast -- For Trondheim
#https://open-meteo.com/en/docs/historical-forecast-api
#https://open-meteo.com/en/docs/historical-forecast-api?hourly=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,showers,snowfall,snow_depth,pressure_msl,cloud_cover,visibility,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m,temperature_80m,temperature_120m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,wind_gusts_10m_max,uv_index_clear_sky_max,uv_index_max,sunshine_duration,daylight_duration,sunrise,sunset,rain_sum,showers_sum,snowfall_sum,precipitation_sum,precipitation_hours&latitude=63.4305&longitude=10.3951&timezone=Europe%2FBerlin
'''
#Installer forecast greiene
pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas
'''

#Imports
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 63.4305,
	"longitude": 10.3951,
	"start_date": "2025-03-09",
	"end_date": "2025-03-23",
	"daily": ["temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "wind_speed_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "wind_gusts_10m_max", "uv_index_clear_sky_max", "uv_index_max", "sunshine_duration", "daylight_duration", "sunrise", "sunset", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_sum", "precipitation_hours"],
	"hourly": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "rain", "showers", "snowfall", "snow_depth", "pressure_msl", "cloud_cover", "visibility", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m", "temperature_80m", "temperature_120m", "temperature_180m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_temperature_18cm", "soil_temperature_54cm", "soil_moisture_0_to_1cm", "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", "soil_moisture_27_to_81cm"],
    "timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
print(f'Sted: Trondheim')

def hourly_results(response):
	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
	hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
	hourly_rain = hourly.Variables(4).ValuesAsNumpy()
	hourly_showers = hourly.Variables(5).ValuesAsNumpy()
	hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
	hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
	hourly_pressure_msl = hourly.Variables(8).ValuesAsNumpy()
	hourly_cloud_cover = hourly.Variables(9).ValuesAsNumpy()
	hourly_visibility = hourly.Variables(10).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(11).ValuesAsNumpy()
	hourly_wind_speed_80m = hourly.Variables(12).ValuesAsNumpy()
	hourly_wind_speed_120m = hourly.Variables(13).ValuesAsNumpy()
	hourly_wind_speed_180m = hourly.Variables(14).ValuesAsNumpy()
	hourly_wind_direction_10m = hourly.Variables(15).ValuesAsNumpy()
	hourly_wind_direction_80m = hourly.Variables(16).ValuesAsNumpy()
	hourly_wind_direction_120m = hourly.Variables(17).ValuesAsNumpy()
	hourly_wind_direction_180m = hourly.Variables(18).ValuesAsNumpy()
	hourly_wind_gusts_10m = hourly.Variables(19).ValuesAsNumpy()
	hourly_temperature_80m = hourly.Variables(20).ValuesAsNumpy()
	hourly_temperature_120m = hourly.Variables(21).ValuesAsNumpy()
	hourly_temperature_180m = hourly.Variables(22).ValuesAsNumpy()
	hourly_soil_temperature_0cm = hourly.Variables(23).ValuesAsNumpy()
	hourly_soil_temperature_6cm = hourly.Variables(24).ValuesAsNumpy()
	hourly_soil_temperature_18cm = hourly.Variables(25).ValuesAsNumpy()
	hourly_soil_temperature_54cm = hourly.Variables(26).ValuesAsNumpy()
	hourly_soil_moisture_0_to_1cm = hourly.Variables(27).ValuesAsNumpy()
	hourly_soil_moisture_1_to_3cm = hourly.Variables(28).ValuesAsNumpy()
	hourly_soil_moisture_3_to_9cm = hourly.Variables(29).ValuesAsNumpy()
	hourly_soil_moisture_9_to_27cm = hourly.Variables(30).ValuesAsNumpy()
	hourly_soil_moisture_27_to_81cm = hourly.Variables(31).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}

	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["apparent_temperature"] = hourly_apparent_temperature
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["rain"] = hourly_rain
	hourly_data["showers"] = hourly_showers
	hourly_data["snowfall"] = hourly_snowfall
	hourly_data["snow_depth"] = hourly_snow_depth
	hourly_data["pressure_msl"] = hourly_pressure_msl
	hourly_data["cloud_cover"] = hourly_cloud_cover
	hourly_data["visibility"] = hourly_visibility
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
	hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
	hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
	hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
	hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
	hourly_data["wind_direction_80m"] = hourly_wind_direction_80m
	hourly_data["wind_direction_120m"] = hourly_wind_direction_120m
	hourly_data["wind_direction_180m"] = hourly_wind_direction_180m
	hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
	hourly_data["temperature_80m"] = hourly_temperature_80m
	hourly_data["temperature_120m"] = hourly_temperature_120m
	hourly_data["temperature_180m"] = hourly_temperature_180m
	hourly_data["soil_temperature_0cm"] = hourly_soil_temperature_0cm
	hourly_data["soil_temperature_6cm"] = hourly_soil_temperature_6cm
	hourly_data["soil_temperature_18cm"] = hourly_soil_temperature_18cm
	hourly_data["soil_temperature_54cm"] = hourly_soil_temperature_54cm
	hourly_data["soil_moisture_0_to_1cm"] = hourly_soil_moisture_0_to_1cm
	hourly_data["soil_moisture_1_to_3cm"] = hourly_soil_moisture_1_to_3cm
	hourly_data["soil_moisture_3_to_9cm"] = hourly_soil_moisture_3_to_9cm
	hourly_data["soil_moisture_9_to_27cm"] = hourly_soil_moisture_9_to_27cm
	hourly_data["soil_moisture_27_to_81cm"] = hourly_soil_moisture_27_to_81cm

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	print(hourly_dataframe)

def daily_results(response):
	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_apparent_temperature_max = daily.Variables(2).ValuesAsNumpy()
	daily_apparent_temperature_min = daily.Variables(3).ValuesAsNumpy()
	daily_wind_speed_10m_max = daily.Variables(4).ValuesAsNumpy()
	daily_wind_direction_10m_dominant = daily.Variables(5).ValuesAsNumpy()
	daily_shortwave_radiation_sum = daily.Variables(6).ValuesAsNumpy()
	daily_wind_gusts_10m_max = daily.Variables(7).ValuesAsNumpy()
	daily_uv_index_clear_sky_max = daily.Variables(8).ValuesAsNumpy()
	daily_uv_index_max = daily.Variables(9).ValuesAsNumpy()
	daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
	daily_daylight_duration = daily.Variables(11).ValuesAsNumpy()
	daily_sunrise = daily.Variables(12).ValuesAsNumpy()
	daily_sunset = daily.Variables(13).ValuesAsNumpy()
	daily_rain_sum = daily.Variables(14).ValuesAsNumpy()
	daily_showers_sum = daily.Variables(15).ValuesAsNumpy()
	daily_snowfall_sum = daily.Variables(16).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(17).ValuesAsNumpy()
	daily_precipitation_hours = daily.Variables(18).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}

	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
	daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
	daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
	daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
	daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
	daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
	daily_data["uv_index_clear_sky_max"] = daily_uv_index_clear_sky_max
	daily_data["uv_index_max"] = daily_uv_index_max
	daily_data["sunshine_duration"] = daily_sunshine_duration
	daily_data["daylight_duration"] = daily_daylight_duration
	daily_data["sunrise"] = daily_sunrise
	daily_data["sunset"] = daily_sunset
	daily_data["rain_sum"] = daily_rain_sum
	daily_data["showers_sum"] = daily_showers_sum
	daily_data["snowfall_sum"] = daily_snowfall_sum
	daily_data["precipitation_sum"] = daily_precipitation_sum
	daily_data["precipitation_hours"] = daily_precipitation_hours

	daily_dataframe = pd.DataFrame(data = daily_data)
	print(daily_dataframe)




Coordinates 63.42746353149414°N 10.403427124023438°E
Elevation 12.0 m asl
Timezone b'Europe/Berlin'b'GMT+1'
Timezone difference to GMT+0 3600 s
Sted: Trondheim
                         date  temperature_2m  relative_humidity_2m  \
0   2025-03-08 23:00:00+00:00        2.765000                  91.0   
1   2025-03-09 00:00:00+00:00        2.815000                  88.0   
2   2025-03-09 01:00:00+00:00        2.865000                  88.0   
3   2025-03-09 02:00:00+00:00        2.915000                  86.0   
4   2025-03-09 03:00:00+00:00        3.065000                  86.0   
..                        ...             ...                   ...   
355 2025-03-23 18:00:00+00:00        9.065000                  48.0   
356 2025-03-23 19:00:00+00:00        8.214999                  51.0   
357 2025-03-23 20:00:00+00:00        7.765000                  54.0   
358 2025-03-23 21:00:00+00:00        7.465000                  54.0   
359 2025-03-23 22:00:00+00:00        7.015000              